In [3]:
from collections import defaultdict,namedtuple

import heapq
import numpy as np

In [12]:
Point = namedtuple("Point", ["x", "y"])

class Grid:
    def __init__(self, grid, mult=1):
        self.grid = grid
        self.h = self.grid.shape[0]
        self.w = self.grid.shape[1]
        self.mult = mult
        self.start = Point(0, 0)
        self.end   = Point(self.w*self.mult-1, self.h*self.mult-1)
    
    @classmethod
    def from_file(cls, filename, mult=1):
        data = []
        with open(filename, "r") as fh:
            for line in fh:
                data.append([int(c) for c in line.strip()])
        return Grid(np.array(data, dtype=int), mult=mult)

    def get(self, p):
        inc = p.x // self.w + p.y // self.h
        value = self.grid[(p.y % self.h),(p.x % self.w)] + inc
        if value > 9:
            return value % 10 + 1
        else:
            return value
    
    def neighbors(self, p):
        for dx, dy in [(-1,0), (0,1), (1,0), (0,-1)]:
            pn = Point(p.x+dx, p.y+dy)
            if pn.x >= 0 and pn.x < self.w*self.mult and pn.y >= 0 and pn.y < self.h*self.mult:
                yield pn
    
    def display(self, path=set()):
        out = ""
        for j in range(self.h*self.mult):
            for i in range(self.w*self.mult):
                p = Point(i, j)
                if p in path:
                    out += "."
                else:
                    out += str(self.get(p))
            out += "\n"
        return out
    
test = Grid.from_file("test.txt")
print(list(test.neighbors(Point(0,0))))
print(list(test.neighbors(Point(2,2))))
print(test.display())

[Point(x=0, y=1), Point(x=1, y=0)]
[Point(x=1, y=2), Point(x=2, y=3), Point(x=3, y=2), Point(x=2, y=1)]
1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581



In [13]:
test5 = Grid.from_file("test.txt", mult=5)
print(test5.get(Point(7, 0)))
print(test5.get(Point(17, 0)))
print(test5.get(Point(27, 0)))
print(test5.get(Point(37, 0)))
print(test5.get(Point(47, 0)))
print("|---------" * 5)
print(test5.display())

7
8
9
1
2
|---------|---------|---------|---------|---------
11637517422274862853338597396444961841755517295286
13813736722492484783351359589446246169155735727126
21365113283247622439435873354154698446526571955763
36949315694715142671582625378269373648937148475914
74634171118574528222968563933317967414442817852555
13191281372421239248353234135946434524615754563572
13599124212461123532357223464346833457545794456865
31254216394236532741534764385264587549637569865174
12931385212314249632342535174345364628545647573965
23119445813422155692453326671356443778246755488935
22748628533385973964449618417555172952866628316397
24924847833513595894462461691557357271266846838237
32476224394358733541546984465265719557637682166874
47151426715826253782693736489371484759148259586125
85745282229685639333179674144428178525553928963666
24212392483532341359464345246157545635726865674683
24611235323572234643468334575457944568656815567976
42365327415347643852645875496375698651748671976285
231424963234253517434

In [14]:
def dijkstra(grid, start):
    visited = set()
    prev = dict()
    pq = []
    costs = defaultdict(lambda: float('inf'))
    costs[start] = 0
    heapq.heappush(pq, (0, start))
    
    while pq:
        curr_cost, curr_node = heapq.heappop(pq)
        visited.add(curr_node)
        
        for next_node in grid.neighbors(curr_node):
            if next_node in visited:
                continue

            next_cost = costs[curr_node] + grid.get(next_node)
            if next_cost < costs[next_node]:
                prev[next_node] = curr_node
                costs[next_node] = next_cost
                heapq.heappush(pq, (next_cost, next_node))
    
    return prev, costs

def get_shortest_path(grid, start, end):
    prev, costs = dijkstra(grid, start)

    path = []
    curr = end
    while curr in prev:
        path.append(curr)
        curr = prev[curr]
    path.append(curr)
    
    #for n in reversed(path):
    #    print(n)
    
    return costs[end], list(reversed(path))
        
trisk, tpath = get_shortest_path(test, Point(0,0), test.end)
print(test.display(tpath))
trisk

.163751742
.381373672
.......328
369493..69
7463417.11
1319128..7
13599124.1
31254216.9
12931385..
231194458.



40

In [15]:
t5risk, t5path = get_shortest_path(test5, test5.start, test5.end)
print(test5.display(t5path))
t5risk

.1637517422274862853338597396444961841755517295286
.3813736722492484783351359589446246169155735727126
.1365113283247622439435873354154698446526571955763
.6949315694715142671582625378269373648937148475914
.4634171118574528222968563933317967414442817852555
.3191281372421239248353234135946434524615754563572
.3599124212461123532357223464346833457545794456865
.1254216394236532741534764385264587549637569865174
.2931385212314249632342535174345364628545647573965
.3119445813422155692453326671356443778246755488935
.2748628533385973964449618417555172952866628316397
.4924847833513595894462461691557357271266846838237
...76224394358733541546984465265719557637682166874
47.51426715826253782693736489371484759148259586125
85.45282229685639333179674144428178525553928963666
24..2392483532341359464345246157545635726865674683
246.......3572234643468334575457944568656815567976
423653274.5347643852645875496375698651748671976285
231424963....5351743453646285456475739656758684176
342215569245...6671356443778246

315

In [23]:
inp = Grid.from_file("input.txt")
get_shortest_path(inp, inp.start, inp.end)[0]

581

In [27]:
inp5 = Grid.from_file("input.txt", mult=5)
get_shortest_path(inp5, inp5.start, inp5.end)[0]

2916